In [1]:
import sys
import numpy as np
import torch
sys.path.append("../")  
import src.RBM as rbm

In [2]:
visible_dim = 327736 
hidden_dim = 1000 
sigma = 0.1
RBM =  rbm.GaussianBinaryRBM(visible_dim, hidden_dim, sigma)

In [3]:
print(RBM)

GaussianBinaryRBM()


In [4]:
v0 = np.random.uniform(-1, 1, visible_dim)
v0 = torch.tensor(v0, dtype=torch.float32)

In [5]:
epoch = 100
for _ in range(epoch):
    RBM.contrastive_divergence(v0)

In [6]:
v_pred = RBM.forward(v0)

In [7]:
diff_norm = torch.norm(v_pred - v0, p=2).item()
print(diff_norm)

112.22213745117188
